### **SETUP:**

In [1]:
!pip install biosppy

     |████████████████████████████████| 85 kB 2.5 MB/s 
  Created wheel for biosppy: filename=biosppy-0.7.3-py2.py3-none-any.whl size=95430 sha256=f058237e4f0a37088d1df17049aa8464556b5f1b764ec302b7ba8d865b9fb86a
  Stored in directory: /root/.cache/pip/wheels/2f/4f/8f/28b2adc462d7e37245507324f4817ce1c64ef2464f099f4f0b
Successfully built biosppy


In [2]:
import matplotlib.pyplot as plt
import biosppy 
from biosppy.signals import ecg
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib
import tensorflow as tf
from tensorflow.keras import utils
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import Input, Model, layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras import losses
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils import class_weight
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier
import math
from scipy.stats import kurtosis
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from scipy.signal import butter, lfilter, welch
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, accuracy_score
from keras.callbacks import ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler, RobustScaler
import pandas as pd
import scipy.io as sio
from os import listdir
from os.path import isfile, join
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Conv1D, GlobalAveragePooling1D, MaxPooling1D
from keras import regularizers
from keras.utils import np_utils



### **PLOT ECG**

In [3]:
def plot_data(signal):
    # Some matplotlib setting 
    plt.rcParams["figure.figsize"] = (150, 15)
    plt.rcParams['lines.linewidth'] = 5
    plt.rcParams['xtick.labelsize'] = 24
    plt.rcParams['ytick.labelsize'] = 32
    plt.rcParams['axes.labelsize'] = 48
    plt.rcParams['axes.titlesize'] = 48

    fig, axs = plt.subplots(1, 1)
    seconds = np.arange(0, 600) / 30 

    # Get a subsequence of a signal and downsample it for visualization purposes
    measurements = signal[1000:7000:10] 
    # convert volts to millivolts
    measurements /= 1000
    axs.plot(seconds, measurements, color='k', rasterized=True)

    # hide tick and tick label of the big axes
    axs.spines['top'].set_visible(False)
    axs.spines['right'].set_visible(False)
    axs.spines['bottom'].set_visible(False)
    axs.spines['left'].set_visible(False)
    plt.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
           
    plt.show()

### **UTILS ON FILES AND DATA:**

In [4]:
def write_X_to_file(file_name, X):
  f  = open(file_name, "w")
  s = "id,"
  for i in range(len(X[0])):
    s = s + "x" + str(i) + ","
  s = s[:-1] + "\n"
  f.write(s)
  
  id = 0
  for x in X:
    list_string = ','.join(str(feature) for feature in x)
    f.write(str(id) + "," + list_string + "\n")
    id = id + 1
  f.close()

def write_y_to_file(file_name, Y):
  f  = open(file_name, "w")
  f.write("id,y\n")
  id = 0
  for y in Y:
    f.write(str(id) + "," + str(y) + "\n")
    id = id + 1
  f.close()

def write_output(file_name, y):
  write_y_to_file(file_name, y)

def write_probs_to_file(file_name, y):
  f  = open(file_name, "w")
  f.write("id,y\n")
  id = 0
  for y in Y:
    f.write(str(id) + "," + str(y) + "\n")
    id = id + 1
  f.close()

In [25]:
def get_train_data():
   
    X = pd.read_csv('drive/MyDrive/AML_TASK_2/X_train.csv', float_precision='high').drop('id', axis=1)
    y = pd.read_csv('drive/MyDrive/AML_TASK_2/y_train.csv', float_precision='high').drop('id', axis=1)

    return X, y

def get_test_data():
   
    X = pd.read_csv('drive/MyDrive/AML_TASK_2/X_test.csv', float_precision='high').drop('id', axis=1)

    return X

def get_processed_train_data():
   
    X = pd.read_csv('drive/MyDrive/AML_TASK_2/X_features_train_puliti.csv', float_precision='high').drop('id', axis=1)
    y = pd.read_csv('drive/MyDrive/AML_TASK_2/y_train.csv', float_precision='high').drop('id', axis=1)

    return X, y

def get_processed_test_data():
   
    X = pd.read_csv('drive/MyDrive/AML_TASK_2/X_features_test_puliti.csv', float_precision='high').drop('id', axis=1)
  
    return X


In [6]:
def split_data_train_test(X, y, test_split=0.1):
    np.random.seed(54678)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_split, shuffle=True)

    return X_train, X_test, y_train, y_test

def split_data_train_test_validation(X, y, test_split=0.15, val_split=0.1):
    np.random.seed(90876)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_split, shuffle=True)

    X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_split, shuffle=True)

    return np.array(X_train), np.array(X_test), np.array(X_val), np.array(y_train), np.array(y_test), np.array(y_val)

In [7]:
# Utils functions for cascade 
def create_y(class1, class2, y_):
    y = np.copy(y_)
    for i, cl in enumerate(y):
        if cl[0] != class1:
            cl[0] = class2
    return y

def remove_class(x_,y_, class1):
    x,y = np.copy(x_), np.copy(y_)
    mask = np.ones(len(y), dtype=bool)
    for i, cl in enumerate(y):
        if cl[0] == class1:
            mask[i] = False
    return x[mask], y[mask]

def remove_class_test(x_,y_, class1):
    x,y = np.copy(x_), np.copy(y_)
    mask = np.ones(len(y), dtype=bool)
    for i, cl in enumerate(y):
        if cl[0] == class1:
            mask[i] = False
    return x[mask]

def add_empty(y, empty):
    for i in empty:
        y = np.insert(y, i, 0)
    return y


### **UTILS:**

In [8]:
# remove NAN values from data
def no_nan(X):
    X_no_nan = []
    for i in range(len(X)):
        measurement = X.loc[i].dropna().to_numpy()
        X_no_nan.append(measurement)
    X_no_nan = np.array(X_no_nan)

    return(X_no_nan)

In [9]:
# given an array return its min, max, mean, median and std
def properties(arr):
  a = np.amin(arr)
  b = np.amax(arr)
  c = np.mean(arr)
  d = np.median(arr)
  e = np.std(arr)

  return a, b, c, d, e

In [10]:
# process each ECG signal with biosspy
def process_biosppy(X):
    out_filtered = []
    out_templates = []
    r_peaks = []
    heart_rates_ts = []
    heart_rates_values = []
    for i in X:
      sig = ecg.ecg(signal=i, sampling_rate=300., show=False)
      out_filtered.append(sig['filtered'])
      out_templates.append(sig['templates'])
      r_peaks.append(sig['rpeaks'])
      heart_rates_ts.append(sig['heart_rate_ts'])
      heart_rates_values.append(sig['heart_rate'])

    return np.array(out_filtered),  np.array(out_templates), np.array(r_peaks), np.array(heart_rates_ts), np.array(heart_rates_values)

### **FEATURES EXTRACTION:**

In [11]:
def process_heart_rate(heart_rate_ts):

  ts_distances = np.diff(heart_rate_ts)
  second_der = np.diff(ts_distances)

  return ts_distances, second_der

In [12]:
def value_distance_ecg(ecg):
  diff = np.diff(ecg)

  std = np.std(diff)
  kurt = kurtosis(diff)
  mean = np.mean(diff)

  return std, kurt, mean

In [13]:
# detect p, q, r, s peaks for a given template of an ECG scan
def peaks_detection(template):
  P_peaks_values, P_peaks_indx, Q_peaks_values, Q_peaks_indx, R_peaks_values, S_peaks_values, S_peaks_indx, T_peaks_values, T_peaks_indx = [],[],[],[],[],[],[],[],[]
  for a in template:
      
      r_peak_indx = np.where(a==max(a[int(len(a)/4):int(len(a)/2)]))[0][0]
      r_peak_value = a[r_peak_indx]
      R_peaks_values.append(r_peak_value)
      first_half = a[:r_peak_indx]

      q_indx = np.where(a==min(first_half[15:]))[0][0]
      Q_peaks_indx.append(q_indx)
      Q_peaks_values.append(a[q_indx])

      first_half = first_half[:q_indx]
      p_indx = np.where(first_half==max(first_half))[0][0]
      P_peaks_indx.append(p_indx)
      P_peaks_values.append(a[p_indx])

      second_half = a[r_peak_indx+1:] 
      s_indx = np.where(a==min(second_half[0:20]))[0][0]
      S_peaks_indx.append(s_indx)
      S_peaks_values.append(a[s_indx])

      end = a[s_indx+1:]
      t_indx = np.where(a==max(end))[0][0]
      T_peaks_indx.append(t_indx)
      T_peaks_values.append(a[t_indx])

    
  return  np.array(P_peaks_values), np.array(P_peaks_indx), np.array(Q_peaks_values), np.array(Q_peaks_indx), np.array(R_peaks_values), np.array(S_peaks_values), np.array(S_peaks_indx), np.array(T_peaks_values), np.array(T_peaks_indx)

In [14]:
# given a list of templates (3d list), extarct a series of features for each ecg
def feature_extraction(templates):
  features = []
  counter = -1
  for template in templates:
    counter += 1
    feature = []
    
    P_peaks_values, P_peaks_indx, Q_peaks_values, Q_peaks_indx, R_peaks_values, S_peaks_values, S_peaks_indx, T_peaks_values, T_peaks_indx = peaks_detection(template)


    _, _, mean, _, std = properties(P_peaks_values)
    feature.append(std)
    feature.append(std/mean)

    _, _, mean, _, std = properties(Q_peaks_values)
    feature.append(std)
    feature.append(std/mean)

    _, _, mean, _, std = properties(R_peaks_values)
    feature.append(std)
    feature.append(std/mean)

    _, _, mean, _, std = properties(S_peaks_values)
    feature.append(std)
    feature.append(std/mean)

    _, _, mean, _, std = properties(T_peaks_values)
    feature.append(std)
    feature.append(std/mean)

    qrs = S_peaks_indx - Q_peaks_indx
    feature.append(np.mean(qrs))
    feature.append(np.std(qrs))

    pr = P_peaks_indx
    feature.append(np.mean(pr))
    feature.append(np.std(pr))

    hear_rate_ts_distances, second_der = process_heart_rate(heart_rates_ts[counter])
    if (len(hear_rate_ts_distances) == 0):
      feature.append(0)
      feature.append(0)
      feature.append(0)
      feature.append(0)
    else:
      minimum, maximum, _, median, std = properties(hear_rate_ts_distances)
      feature.append(minimum)
      feature.append(maximum)
      feature.append(median)
      feature.append(std)

    if (len(second_der) == 0):
      feature.append(0)
      feature.append(0)
      feature.append(0)
      feature.append(0)
    else:
      minimum, maximum, _, median, std = properties(second_der)
      feature.append(minimum)
      feature.append(maximum)
      feature.append(median)
      feature.append(std)


    if len(heart_rates_values[counter]) == 0:
      feature.append(0)
      feature.append(0)
      feature.append(0)
      feature.append(0)
    else:
      minimum, maximum, _, median, std = properties(heart_rates_values[counter])
      feature.append(minimum)
      feature.append(maximum)
      feature.append(median)
      feature.append(std)

    std, kurt, mean = value_distance_ecg(X_processed[counter])
    feature.append(mean)
    feature.append(std)
    feature.append(kurt)

    
    features.append(feature)
  return features

In [15]:
# Uncomment this if you want to load the train file, extrapolate the features
# and create the feature file

"""
# get train data and create features file
X, y = get_train_data()
X = no_nan(X)
X_processed, templates, r_peaks,  heart_rates_ts, heart_rates_values = process_biosppy(X)
X_processed = np.array(X_processed)
templates = np.array(templates)
r_peaks = np.array(r_peaks)
features = feature_extraction(templates)
write_X_to_file("drive/MyDrive/AML_TASK_2/X_features_train.csv", features)
"""

'\n# get train data and create features file\nX, y = get_train_data()\nX = no_nan(X)\nX_processed, templates, r_peaks,  heart_rates_ts, heart_rates_values = process_biosppy(X)\nX_processed = np.array(X_processed)\ntemplates = np.array(templates)\nr_peaks = np.array(r_peaks)\nfeatures = feature_extraction(templates)\nwrite_X_to_file("drive/MyDrive/AML_TASK_2/X_features_train.csv", features)\n'

In [16]:
# Uncomment this if you want to load the test file, extrapolate the features
# and create the feature file
"""
# get test data and create features file 
X_test = get_test_data()
X_test = no_nan(X_test)
X_processed, templates, r_peaks,  heart_rates_ts, heart_rates_values = process_biosppy(X_test)
X_processed = np.array(X_processed)
templates = np.array(templates)
r_peaks = np.array(r_peaks)
features = feature_extraction(templates)
write_X_to_file("drive/MyDrive/AML_TASK_2/X_features_test.csv", features)
"""

'\n# get test data and create features file \nX_test = get_test_data()\nX_test = no_nan(X_test)\nX_processed, templates, r_peaks,  heart_rates_ts, heart_rates_values = process_biosppy(X_test)\nX_processed = np.array(X_processed)\ntemplates = np.array(templates)\nr_peaks = np.array(r_peaks)\nfeatures = feature_extraction(templates)\nwrite_X_to_file("drive/MyDrive/AML_TASK_2/X_features_test.csv", features)\n'

### **MODELS**

In [27]:
# get the processed train data
X, y = get_processed_train_data()
X = np.array(X)
y = np.array(y)

# get the processed test data
X_TEST = get_processed_test_data()
X_TEST = np.array(X_TEST)

In [ ]:
# split the data into train test and validation
X_train, X_test, X_val, y_train, y_test, y_val = split_data_train_test_validation(X, y)

# compute class weights to account for class imbalance
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train.reshape(-1))
class_weights = {0: class_weights[0],
                 1: class_weights[1],
                 2: class_weights[2],
                 3: class_weights[3]
                 }
print('Training data: class weights {}'.format(class_weights))

Training data: class weights {0: 0.4199570815450644, 1: 3.0673981191222572, 2: 0.8697777777777778, 3: 6.989285714285714}


In [28]:

# perform one hot encoding
n_classes = 4
y = utils.to_categorical(y, n_classes)

# perform standardization
# scaler = StandardScaler().fit(X_train)
# X_train = scaler.transform(X_train)
# X_test = scaler.transform(X_test) 

# create the neural network
model = Sequential()
model.add(Dense(200, activation='relu', input_dim=29))
model.add(Dense(200, activation='relu'))
model.add(Dense(200, activation='relu'))
model.add(Dense(4, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

model.summary()
# fit the model              
# model.fit(X_train, y_train, epochs=100, batch_size=15, verbose=0)

# y_pred = model.predict(X_test)
# y_pred = np.argmax(y_pred, axis=1)

# from sklearn.metrics import f1_score
# F1 = f1_score(y_test, y_pred, average='micro')
# print(F1)

# # predict on the test data
# y_pred = model.predict(X_test)
# y_pred = np.argmax(y_pred, axis=1)

scaler = StandardScaler().fit(X)
X = scaler.transform(X)
X_TEST = scaler.transform(X_TEST)  

# fit the model              
model.fit(X, y, epochs=100, batch_size=15, verbose=0)

y_pred = model.predict(X_TEST)
y_pred_no_prob = np.argmax(y_pred, axis=1)


write_output("drive/MyDrive/AML_TASK_2/nn.csv", y_pred_no_prob)
write_output("drive/MyDrive/AML_TASK_2/nn_probs.csv", y_pred)

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 200)               6000      
                                                                 
 dense_17 (Dense)            (None, 200)               40200     
                                                                 
 dense_18 (Dense)            (None, 200)               40200     
                                                                 
 dense_19 (Dense)            (None, 4)                 804       
                                                                 
Total params: 87,204
Trainable params: 87,204
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from xgboost import XGBClassifier

xgb_model = XGBClassifier()

cv = RepeatedKFold(n_splits=5, n_repeats=3, random_state=1)
n_scores = cross_val_score(xgb_model, X, y, scoring='f1_micro', cv=cv, n_jobs=-1, error_score='raise')
print('MAE: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))

# from sklearn.metrics import f1_score
# F1 = f1_score(y_test, y_pred, average='micro')
# print(F1)

xgb_model.fit(X, y)
y_pred = xgb_model.predict_proba(X_TEST)
write_output("drive/MyDrive/AML_TASK_2/xgb.csv", y_pred)

MAE: 0.802 (0.013)


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
# -------------------------------------------------
# -------------------- CASCADE --------------------
# -------------------------------------------------

## Train the models:
# First: 3 vs rest
y_3_rest = create_y(3, 4, y)
gbc_3_rest = GradientBoostingClassifier(n_estimators=250, learning_rate=0.2, random_state=0)
gbc_3_rest.fit(X, y_3_rest)

# Second: 0 vs 1,2
x_3, y_3 = remove_class(X, y, 3)
y_0_rest = create_y(0, 4, y_3)
gbc_0_rest = GradientBoostingClassifier(n_estimators=250, learning_rate=0.2, random_state=0)
gbc_0_rest.fit(x_3, y_0_rest)

# Third : 1 vs 2
x_03, y_03 = remove_class(x_3, y_3, 0)
y_1_2 = create_y(1, 4, y_03)
gbc_1_2 = GradientBoostingClassifier(n_estimators=250, learning_rate=0.2, random_state=0)
gbc_1_2.fit(x_03, y_1_2)


# Predict TEST using cascade 
y_pred = np.ones(len(X_TEST), dtype=int)
for i, x in enumerate(X_TEST):
    prediction = gbc_3_rest.predict([x])
    
    if prediction == 3:
        y_pred[i] = 3
        continue
    
    prediction = gbc_0_rest.predict([x])
    if prediction == 0:
        y_pred[i] = 0
        continue
    
    prediction = gbc_1_2.predict([x])
    if prediction == 1: 
      y_pred[i] = 1
    else:
      y_pred[i] = 2

write_output("drive/MyDrive/AML_TASK_2/cascade.csv", y_pred)

In [ ]:
# ------------------------------------------------
# -------------------- FOREST --------------------
# ------------------------------------------------

# Create the model with 100 trees
model = RandomForestClassifier(n_estimators=150)
"""
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
n_scores = cross_val_score(model, X, y, scoring='f1_micro', cv=cv, n_jobs=-1, error_score='raise')
print('MAE: %.3f (%.3f)' % (np.mean(n_scores), np.std(n_scores)))
"""

# Uncomment this if you want to train the forest on the entire train data set
# and create the output file for the REAL test data
# perform one hot encoding
# n_classes = 4
# y_train = utils.to_categorical(y, n_classes)
model.fit(X, y)
# predict on the test data
y_pred = model.predict_proba(X_TEST)
# print(y_pred)

y_pred_new = []
for i in y_pred:
  inner = []
  for j in i:
    j = float(j)
    inner.append(j)
  y_pred_new.append(inner)

write_output("drive/MyDrive/AML_TASK_2/forest_probs.csv", y_pred_new)




/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


### **CONVOLUTIONAL NEURAL NETWORK**

In [ ]:
# get train data
X, y = get_train_data()
X = no_nan(X)
X, templates, r_peaks,  heart_rates_ts, heart_rates_values = process_biosppy(X)
X = np.array(X)
y = np.array(y)

In [ ]:
# standardize it
X_new = []
for  x in X:
  mean_len += len(x)
  x = (x - np.mean(x)) / np.std(x)
  X_new.append(x)
X_new = np.array(X_new)

# pad with 0s
signal_lengths = [len(signal) for signal in X_new]
max_length = np.amax(signal_lengths)
X_new_new = []
for x in X_new:
  padding = max_length - len(x)
  x = np.hstack((x, [0] * padding))
  X_new_new.append(x)

X_new_new = np.array(X_new_new)

In [ ]:
# get test data
X_test = get_test_data()
X_test = no_nan(X_test)
X_test, templates, r_peaks,  heart_rates_ts, heart_rates_values = process_biosppy(X_test)
X_test = np.array(X_test)

In [ ]:
# standardize it
X_new_test = []
for x in X_test:
  x = (x - np.mean(x)) / np.std(x)
  X_new_test.append(x)
X_new_test = np.array(X_new_test)

# pad with 0s
X_new_new_test = []
for x in X_new_test:
  padding = 17841 - len(x)
  x = np.hstack((x, [0] * padding))
  X_new_new_test.append(x)

X_new_new_test = np.array(X_new_new_test)

In [ ]:
# --------------------------------------------------------
# ------------------------- CNN -------------------------
# --------------------------------------------------------

# perform one hot encoding
n_classes = 4
y_train = utils.to_categorical(y, n_classes)

# create the model
model = Sequential()
model.add(Conv1D(128, 55, activation='relu', input_shape=(17841, 1)))
model.add(MaxPooling1D(10))
model.add(Dropout(0.5))
model.add(Conv1D(128, 25, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Dropout(0.5))
model.add(Conv1D(128, 10, activation='relu'))
model.add(MaxPooling1D(5))
model.add(Dropout(0.5))
model.add(Conv1D(128, 5, activation='relu'))
model.add(GlobalAveragePooling1D())
# model.add(Flatten())
model.add(Dense(256, kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, kernel_initializer='normal', activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4, kernel_initializer='normal', activation='softmax'))

model.summary()

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_new_new, y_train, batch_size=235, epochs=50, verbose=2, shuffle=True)

y_pred = model.predict(X_new_new_test)
y_pred_no_prob = np.argmax(y_pred, axis=1)

write_output("drive/MyDrive/AML_TASK_2/cnn.csv", y_pred_no_prob)
write_output("drive/MyDrive/AML_TASK_2/cnn_probs.csv", y_pred)

In [ ]:
# Perform a voting --> use the predictions from different models to create a new
# set of predictions
def democracy(file_names):

  f_output = open("drive/MyDrive/AML_TASK_2/democracy.csv", "w")
  files = []
  # opene all the files passed as inputs
  for file_name in file_names:
      f = open(file_name, "r")
      files.append(f)
  
  # discard all headers
  for f in files:
      header = f.readline()
  f_output.write(header)

  for _ in range(3411):
      readings = []
      id = 0
      class_counter = [0] * 4
      for f in files:
          line = f.readline()

          id = line.split(',')[0]
          reading = int(line.split(',')[1])
          class_counter[reading] = class_counter[reading] + 1

      dem = class_counter.index(max(class_counter))

      f_output.write(id + "," + str(dem) + "\n")

  for f in files:
      f.close()
  
  f_output.close()

# Perform a voting --> use the probabilities of the predictions from different
# models to create a new set of predictions
def democracy_probs(file_names):

  f_output = open("drive/MyDrive/AML_TASK_2/democracy_probs.csv", "w")
  files = []
  # opene all the files passed as inputs
  for file_name in file_names:
      f = open(file_name, "r")
      files.append(f)
  
  # discard all headers
  for f in files:
      header = f.readline()
  f_output.write(header)

  for _ in range(3411):
      readings = []
      id = 0

      total_probs = [0] * 4
      for i, f in enumerate(files):
          line = f.readline()

          id = line.split(',')[0]
          if i == 1:
            reading = line.split(',')[1:]
          else:
            reading = line.split(',')[1]
            reading = reading.split(' ')
          reading =  [x for x in reading if x != '']
          reading[0] = (reading[0])[1:]
          reading[3] = (reading[3].strip())[:-1]
          
          if (len(reading) > 4):
            reading = reading[:4]


          # reading = list(filter('', reading))
          # print(reading)
          
          
          reading = np.array(reading, dtype=np.float32)
          
          
          if i == 3:
            reading = 1/2 * reading
          else:
            reading = 1/6 * reading
          total_probs += reading
          # print(total_probs)

      dem = np.argmax(total_probs)
      # print(dem)
      f_output.write(id + "," + str(dem) + "\n")

  for f in files:
      f.close()
  
  f_output.close()



In [ ]:
# This is vai democracy works
# file_names = ['drive/MyDrive/AML_TASK_2/cascade.csv', 'drive/MyDrive/AML_TASK_2/forest.csv', 'drive/MyDrive/AML_TASK_2/mlp.csv', 'drive/MyDrive/AML_TASK_2/cnn.csv', 'drive/MyDrive/AML_TASK_2/svc.csv']
# democracy(file_names)

file_names = ['drive/MyDrive/AML_TASK_2/nn_probs.csv', 'drive/MyDrive/AML_TASK_2/forest_probs.csv', 'drive/MyDrive/AML_TASK_2/xgb_probs.csv', 'drive/MyDrive/AML_TASK_2/cnn_probs_padding.csv']
democracy_probs(file_names)